<a href="https://colab.research.google.com/github/Sep-eg/kagglestudy/blob/main/titanic/BCT_Lv_1_1st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🚢  Titanic Tutorial 1
이 노트북은 {AIB Alumni Challenge - Season2: 캐글 필사 스터디}를 위해 작성하였습니다.

코드의 출처는 [Kaggle-KR 블로그](https://kaggle-kr.tistory.com/17?category=868316)로 클릭하면 이동할 수 있습니다.




In [ ]:
!pip install kaggle # kaggle API를 사용하기위해 설치

In [ ]:
# kaggle API를 사용하기 위해서는 "root/.kaggle/"경로에 API key가 담긴 kaggle.json파일이 들어가야한다.
!mkdir -p ~/.kaggle  # root디렉토리에 .kaggle디렉토리 생성(mkdir -p "경로")
!cp kaggle.json ~/.kaggle/ # 업로드한 kaggle.json 파일을 .kaggle디렉토리로 복사(cp "복사하는 파일명" "붙여넣을 경로")

In [ ]:
!kaggle competitions download -c titanic

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn') # 생서할 그래프의 스타일시트를 변경 ggplot, grayscale, seaborn 등 다양한 파라미터 사용가능
sns.set(font_scale=2.5) # 앞으로 생성할 모든 그래프의 폰트 사이즈를 지정

import missingno as msno # 데이터의 결측치를 보기 쉽게 보여주는 시각화 도구

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 1.데이터셋 확인
가장 먼저 분석에 사용할 데이터가 어떤 데이터인지 확인하는 작업을 거칩니다.

In [ ]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train.head()
# Cabin column에 NaN값이 확인됩니다.

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

###1.1 Null data check

In [ ]:
# Null data를 확인하기위해 for문을 통해 column별로 연산을 진행합니다.
for col in df_train.columns:
    msg = 'column: {:>11}\t Percent of NaN value: {:.2f}%'.format(
        col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

In [ ]:
for col in df_test.columns:
    msg = 'column: {:>11}\t Percent of NaN value: {:.2f}%'.format(
        col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

In [ ]:
# missingno라이브러리를 활용하면 조금 더 보기편하게 확인할 수 있습니다.
msno.matrix(df=df_train, figsize=(8, 8), color=(0.8, 0.5, 0.2))

In [ ]:
msno.bar(df=df_train.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2))

In [ ]:
msno.bar(df=df_test.iloc[:,:], figsize=(8,8), color=(0.8, 0.5, 0.2))

###1.2 Target label 확인
다음으론 타겟값이 어떤 분포를 가지고 있는지 확인합니다.

이진 분류 문제의 경우, 1과 0의 분포에 따라 모델의 평가 방법이 달라 질 수 있습니다.

만약, 100개의 데이터 중 1개만 1, 나머지 99개가 0의 분포를 가진다면 0으로만 예측해도 정확도가 99%로 나올 수 있기 때문입니다.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18,8))

df_train['Survived'].value_counts().plot.pie(
    explode=[0, 0.1], autopct="%1.1f%%", ax=ax[0], shadow=True)
ax[0].set_title('Pie Plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()
# 타겟이 어느정도 균일한 분포를 보이는 것을 확인할 수 있습니다.

## 2.EDA(Exploratory Data Analysis)

###2.1 Pclass
Pclass는 탑승객의 좌석등급을 나타내는 특성입니다. ordinal, 서수형 데이터로 카테고리이면서, 순서가 있는 데이터 타입입니다.

In [ ]:
# Pclass를 기준으로 데이터를 묶고 각 클래스별 데이터의 갯수를 카운트 합니다.
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).count()

In [ ]:
# Survived가 0과 1로 이루어진것을 이용하여 각 클래스별 분류된 데이터를 합합니다.
# 1은 생존자로 생존자의 총합을 알 수 있습니다.
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).sum()

In [ ]:
# pandas의 crosstab을 이용해 간편하게 확인할 수 있습니다.
pd.crosstab(df_train["Pclass"], df_train['Survived'],
            margins=True).style.background_gradient(cmap='summer_r')
# margins는 테이블의 각 열과 행의 합을 표시해줍니다.

In [ ]:
df_train[['Pclass','Survived']].groupby(['Pclass'],as_index=True).mean(
).sort_values(by='Survived', ascending=False).plot.bar()

Pclass 등급이 높을 수록 생존자 비율이 높은 것을 볼 수 있습니다.

In [ ]:
y_position = 1.02
f, ax = plt.subplots(1, 2, figsize=(20, 8))
df_train['Pclass'].value_counts().plot.bar(
    color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of Passenger by Pclass', y=y_position)
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)
plt.show()

Pclass 등급이 높을수록 생존률이 높은 것을 확인할 수 있고,
이는 생존에 Pclass가 큰 영향을 미친다고 판단할 수 있게 해줍니다.

###2.2 Sex
Sex는 탑승객의 성별을 나타내는 특성입니다.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(20, 8))
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')
plt.show()

여성의 생존 비율이 남성에 비해 매우 높은것을 확인할 수 있습니다.

In [ ]:
df_train[['Sex', 'Survived']].groupby(['Sex'],as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

Sex역시 Pclass와 같이 중요한 특성임을 알 수 있습니다.

###2.3 Both Sex and Pclass
Sex와 Pclass, 두가지 특성을 이용하여 Survived가 어떻게 달라지는지 확인해 봅니다.

In [ ]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=df_train,
               size=6, aspect=1.5) # aspect는 그래프의 너비 / 높이 를 나타냅니다. 클수록 그래프가 넓고 낮아집니다.

모든 클래스에서 남성보다 여성의 생존률이 높으며, 남여 모두 클래스가 높아질수록 생존률이 높아집니다.

In [ ]:
sns.factorplot(x='Sex', y='Survived', col='Pclass', data=df_train,
               satureation=.5, size=9, aspect=1)

### 2.4 Age
Age는 탑승객의 나이를 나타내는 특성입니다.

In [ ]:
print('제일 나이 많은 탑승객 : {:.1f} Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=ax)
plt.legend(['Survived == 1', 'Survivd == 0'])
plt.show()

나이가 어린 탑승객이 생존자의 비율이 높은것을 볼 수 있습니다.

In [ ]:
plt.figure(figsize=(8, 6))
df_train['Age'][df_train['Pclass'] == 1].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 2].plot(kind='kde')
df_train['Age'][df_train['Pclass'] == 3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distibution within classes')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

클래스가 높을 수록 나이가 많은 탑승객의 비중이 커지는 것을 알 수 있습니다.

In [ ]:
# 나이가 증가함에 따라 생존률의 변화를 살펴봅니다.
cummulate_survival_ratio = []
for i in range(1,80):
    cummulate_survival_ratio.append(df_train[df_train['Age'] < i]['Survived'].sum() /
                                    len(df_train[df_train['Age'] < i]['Survived']))
    
plt.figure(figsize=(7, 7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

나이가 많아질수록 생존자 비율이 확연히 줄어드는 것을 확인할 수 있습니다.

###2.5 Pclass, Sex, Age
Sex, Pclass, Age를 모두 활용해 Survived에 대해 살펴봅니다.

In [ ]:
f, ax=plt.subplots(1,2,figsize=(20,8))
sns.violinplot("Pclass","Age", hue="Survived", data=df_train,
               scale="count", split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age", hue="Survived", data=df_train,
               scale='count', split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

모든 클래스에 대해 나이가 어릴수록 그리고 성별이 여성일 경우 생존률이 높을것을 다시 확인 할 수 있었습니다.

###2.6 Embarked
Embarked는 탑승객이 탑승한 항구를 나타냅니다.

In [ ]:
f, ax = plt.subplots(1,1, figsize=(7,7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'],as_index=True
                                           ).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax)

탑승객 클래스, 성별 등에 비해 큰 차이를 보이지는 않는것으로 보입니다.

다른 특성들과 split하여 살펴보도록 하겠습니다.

In [ ]:
f,ax=plt.subplots(2, 2, figsize=(20,15))
sns.countplot('Embarked', data=df_train, ax=ax[0,0])
ax[0,0].set_title('(1) No. Of Passengers Boarded')
sns.countplot('Embarked', hue='Sex', data=df_train, ax=ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived', data=df_train, ax=ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass', data=df_train, ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

(1) 탑승객의 숫자는 S, C, Q순으로 많은것으로 나타났습니다.

(2) C, Q는 남여 탑승객의 숫자가 비슷하지만, S는 남성이 확연히 많은것으로 나타났습니다.

(3) S, Q는 C에 비해 생존자 비율이 낮은것으로 나타났습니다.

(4) C는 다른 클래스에 비해 1st 클래스 탑승자 비율이 높고, S는 3rd 클래스 탑승자가 많은것으로 나타났습니다.

###2.7 Family - SibSp(형제, 자매) + Parch(부모, 자녀)
SibSp는 형제, 자매를 Parch는 부모, 자녀를 나타내는 특성으로 합쳐서 Family 특성으로 분석해봅니다.

In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 # 자기자신을 포함하기 위해 1을 더합니다.
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 # 자기자신을 포함하기 위해 1을 더합니다.

In [ ]:
print("Maximum size of Family: ", df_train['FamilySize'].max())
print("Maximum size of Family: ", df_train['FamilySize'].min())

In [ ]:
f, ax=plt.subplots(1, 3, figsize=(40,10))
sns.countplot('FamilySize', data=df_train, ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

sns.countplot('FamilySize', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('(2) Survivd countplot depending on FamilySize', y=1.02)

df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=True
                                             ).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize', y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

(1) 가족크기는 1~11까지 다양하게 나타났으며 대부분 1인 탑승객으로 점차 커질수록 수가 적어지는것으로 나타났습니다.

(2), (3) 다른 크기에 비해 2, 3, 4인으로 이루어진 가족의 생존률이 높으 것을 확인할 수 있습니다.

### 2.8 Fare
Fare는 탑승요금을 나타내는 특성입니다.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax=ax)
g = g.legend(loc='best')

Fare의 분포가 매우 비대칭인 것을 알 수 있습니다. 이대로 데이터를 모델에 넣어줄 경우 모델이 잘못 학습할 수 있습니다.

따라서, outlier의 영향을 줄이기 위해 Fare에 log를 취하도록 하겠습니다.

In [ ]:
df_test.loc[df_test.Fare.isnull(), 'Fare'] = df_test['Fare'].mean() # test데이터의 Fare 결측치를 평균값으로 치환합니다.

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
# map 메소드는 Series타입 변수(1개의 칼럼)를 순환하며 파라미터로 주어진 함수를 적용시키고 리턴합니다.
# apply 메소드는 여러개의 칼럼을 활용해야할 경우 사용할 수 있습니다.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax=ax)
g = g.legend(loc='best')

log를 취하면서 비대칭성이 많이 해소되었습니다.

###2.9 Cabin
Cabin은 객실번호를 나타내는 특성으로, 앞에서 살펴보았을때 Null data의 비율이 80%에 가까워 정보를 얻기 쉽지않습니다. 따라서 모델에는 포함하지 않겠습니다.

In [ ]:
df_train.head()

###2.10 Ticket
Ticket은 탑승권의 번호를 나타내는 특성입니다.

String형태의 데이터로 이를 분석에 활용하기 위해서는 추가적인 아이디어가 필요합니다.

이번 필사과정에서는 생략하고, 추후 활용해보도록 하겠습니다.


In [ ]:
df_train['Ticket'].value_counts()

#🚢  Titanic Tutorial 2
이 노트북은 {AIB Alumni Challenge - Season2: 캐글 필사 스터디}를 위해 작성하였습니다.

코드의 출처는 [Kaggle-KR 블로그](https://kaggle-kr.tistory.com/18?category=8683166)로 클릭하면 이동할 수 있습니다.


In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

## 3.Feature engineering

###3.1 Fill Null

#### 3.1.1 Fill Null in Age using title
Age의 null data 177개를 채우기 위해 영어이름의 title을 사용해봅니다.

In [ ]:
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.')
df_test['Initial'] = df_test.Name.str.extract('([A-Za-z]+)\.')

In [ ]:
# crosstab을 사용하여 Initial과 Sex간의 count를 살펴봅니다.
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')

위의 표를 활용하여 남성, 여성이 사용하는 initial을 구분하여 치환하도록 하겠습니다.

In [ ]:
df_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

df_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

In [ ]:
df_train.groupby('Initial').mean()

여성과 관련된 Miss와 Mrs의 생존률이 높은것을 확인할 수 있습니다.

In [ ]:
df_train.groupby('Initial')['Survived'].mean().plot.bar()

이제 본격적으로 Null data를 채워주기 위한 작업을 진행합니다.

Null data를 채우기위한 많은 방법이 존재하지만, 여기서는 통계량을 활용하여 채우도록 합니다.

이때 통계량은 train data의 값 만을 의미합니다. test data는 학습에 활용해서는 안되기 때문에

test data의 Null data역시 train data를 기반으로 채워주도록 합니다.

In [ ]:
df_train.groupby('Initial').mean()

In [ ]:
# Initial별로 Age의 평균을 활용해 Null data를 치환해 줍니다.
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mr'),'Age'] = 33
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mrs'),'Age'] = 36
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Master'),'Age'] = 5
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Miss'),'Age'] = 22
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Other'),'Age'] = 46

df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mr'),'Age'] = 33
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mrs'),'Age'] = 36
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Master'),'Age'] = 5
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Miss'),'Age'] = 22
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Other'),'Age'] = 46

#### 3.1.2 Fill Null in Embarked
탑승지의 Null data는 2개로 매우 적으므로, 가장 많은 탑승객이 탑승한 'S'를 채워주도록 하겠습니다.

In [ ]:
print('Embarked has ', sum(df_train['Embarked'].isnull()), ' Null values')

In [ ]:
df_train['Embarked'].fillna('S', inplace=True)

### 3.2 Change Age(continuous to categorical)
continuous형태인 Age를 몇개의 그룹으로 나누어 categorical데이터로 변환시켜봅니다.



In [ ]:
# loc메소드를 활용해서 특정 조건을 만족하는 그룹별로 일일이 치환시켜줍니다.
df_train['Age_cat'] = 0
df_train.loc[df_train['Age'] < 10, 'Age_cat'] = 0
df_train.loc[(10 <= df_train['Age']) & (df_train['Age'] < 20), 'Age_cat'] = 1
df_train.loc[(20 <= df_train['Age']) & (df_train['Age'] < 30), 'Age_cat'] = 2
df_train.loc[(30 <= df_train['Age']) & (df_train['Age'] < 40), 'Age_cat'] = 3
df_train.loc[(40 <= df_train['Age']) & (df_train['Age'] < 50), 'Age_cat'] = 4
df_train.loc[(50 <= df_train['Age']) & (df_train['Age'] < 60), 'Age_cat'] = 5
df_train.loc[(60 <= df_train['Age']) & (df_train['Age'] < 70), 'Age_cat'] = 6
df_train.loc[70 <= df_train['Age'], 'Age_cat'] = 7

df_test['Age_cat'] = 0
df_test.loc[df_test['Age'] < 10, 'Age_cat'] = 0
df_test.loc[(10 <= df_test['Age']) & (df_test['Age'] < 20), 'Age_cat'] = 1
df_test.loc[(20 <= df_test['Age']) & (df_test['Age'] < 30), 'Age_cat'] = 2
df_test.loc[(30 <= df_test['Age']) & (df_test['Age'] < 40), 'Age_cat'] = 3
df_test.loc[(40 <= df_test['Age']) & (df_test['Age'] < 50), 'Age_cat'] = 4
df_test.loc[(50 <= df_test['Age']) & (df_test['Age'] < 60), 'Age_cat'] = 5
df_test.loc[(60 <= df_test['Age']) & (df_test['Age'] < 70), 'Age_cat'] = 6
df_test.loc[70 <= df_test['Age'], 'Age_cat'] = 7

In [ ]:
# apply메소드를 활용하여 조건문을 통해 한 번에 치환시켜봅니다.
def category_age(x):
    if x < 10:
        return 0
    elif x < 20:
        return 1
    elif x < 30:
        return 2
    elif x < 40:
        return 3
    elif x < 50:
        return 4
    elif x < 60:
        return 5
    elif x < 70:
        return 6
    else:
        return 7    
    
df_train['Age_cat_2'] = df_train['Age'].apply(category_age)

In [ ]:
print('1번 방법, 2번 방법 둘다 같은 결과를 내면 True -> ', (df_train['Age_cat'] == df_train['Age_cat_2']).all())

In [ ]:
# 두 방법이 같은 결과를 내는것을 확인, 필요없어진 특성은 삭제하도록 하겠습니다.
df_train.drop(['Age', 'Age_cat_2'], axis=1, inplace=True)
df_test.drop(['Age'], axis=1, inplace=True)

### 3.3 Change Initial, Embarked and Sex(string to numerical)
현재 Initial과 Embarked, Sex는 string형태의 데이터로 모델이 학습할 수 없습니다.

이를 모델에 입력할 수 있도록 수치형 categorical데이터로 변환시키겠습니다.

In [ ]:
df_train['Initial'] = df_train['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})
df_test['Initial'] = df_test['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs': 3, 'Other': 4})

In [ ]:
# unique메소드를 활용해서 특성이 어떤 값들로 이루어져 있는지 확인할 수 있습니다.
df_train['Embarked'].unique()

In [ ]:
# value_counts메소드를 활용해서 특성을 이루는 값 들이 몇 개씩 분포하였는지 확인할 수 있습니다.
df_train['Embarked'].value_counts()

In [ ]:
df_train['Embarked'] = df_train['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
df_test['Embarked'] = df_test['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

In [ ]:
# isnull, any 메소드를 활용하면 결측값이 있는지 확인할 수 있습니다.
df_train['Embarked'].isnull().any()

In [ ]:
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})

데이터를 모두 수치형으로 바꿔주었으므로, 각 특성간의 상관관계를 살펴볼 수 있게 되었습니다.

두 변수간의 상관관계는 (-1, 1)사이의 값으로 나타나며 -1로 갈수록 음의 상관관계, 1로 갈수록 양의 상관관계를 의미하며, 0은 상관관계가 없다는 것을 의미합니다.

In [ ]:
# 변수간의 상관관계는 pandas DataFrame의 corr메소드를 사용하면 쉽게 구할수있으며, seaborn heatmap을 이용해 보기 좋게 표현할 수 있습니다.
heatmap_data = df_train[['Survived', 'Pclass', 'Sex', 'Fare', 'Embarked', 'FamilySize', 'Initial', 'Age_cat']] 

colormap = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

del heatmap_data

* EDA를 통해 살펴보았듯, Sex와 Pclass, Sex가 Survived와 상관관계가 어느정도 있는 것을 확인할 수 있습니다.
* Fare와 Embarked또한 다른 특성들에 비해 어느정도 상관관계를 가지는 것을 볼 수 있습니다.
* 서로 매우 강한 상관관계를 가지는 특성들은 없는 것으로 확인됩니다. 이는 모델학습에 불필요한 특성이 없는 것을 나타냅니다.

* 상관관계가 1 혹은 -1을 가진 특성이 있다면 실제로 얻을 수 있는 정보는 하나 뿐이기 때문입니다.

###3.4 One-hot encoding on Initial and Embarked
수치화 시킨 categorical데이터는 그대로 사용 할 수도 있지만, 모델의 성능을 높이기 위해 one-hot encoding을 할수도 있습니다.

In [ ]:
# one-hot encoding은 카테고리를 각각의 특성으로 분리시키는 encoding기법입니다.
# 직접코딩을 할 수도 있지만, pandas의 get_dummies를 활용하면 간편하게 해결 할 수 있습니다.
df_train = pd.get_dummies(df_train, columns=['Initial'], prefix='Initial')
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial')

In [ ]:
df_train.head()

In [ ]:
df_train = pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked')
df_test = pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked')

In [ ]:
df_train.head()

### 3.5 Drop columns
필요한 columns를 제외하고 모두 제거해 줍니다.

In [ ]:
df_train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_test.drop(['PassengerId', 'Name',  'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## 4.Building machine learning model and prediction using the trained model
본격적으로 sklearn 라이브러리를 활용해 모델을 만듭니다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

### 4.1 Preparation - Split dataset into train, valid, test set

In [ ]:
X_train = df_train.drop('Survived', axis=1).values
target_label = df_train['Survived'].values
X_test = df_test.values

In [ ]:
X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, target_label, test_size=0.3, random_state=2018)

###4.2 Model generation and prediction

In [ ]:
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
prediction = model.predict(X_vld)

In [ ]:
print('총 {}명 중 {:.2f}% 정확도로 생존을 맞춤'.format(y_vld.shape[0], 100 * metrics.accuracy_score(prediction, y_vld)))

### 4.3 Feature importance
학습된 모델은 Feature importance를 가지게 됩니다. 이를 확인하여 모델이 어떤 feature에 영향을 많이 받았는지 확인할 수 있습니다.

In [ ]:
feature_importance = model.feature_importances_
Series_feat_imp = pd.Series(feature_importance, index=df_test.columns)

In [ ]:
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

* 학습을 통해 얻은 모델에서는 Fare가 가장 큰 중요도를 가지고 있으며, Sex, Age_cat, initial_2 등이 높은 중요도를 보였습니다.
* 이는 지금 만들어진 모델의 importance로 다른 모델을 사용하면, 같은 데이터를 사용하더라도 달라질 수 있습니다.
* feature importance를 참고하여 더 정확도 높은 모델을 위한 feature selection을 할 수도, 더 빠른 모델을 위해 feature를 제거 할 수도 있습니다.

### 4.4 Prediction on Test set
이제 모델이 학습하지 않은 테스트셋을 모델을 통해 예측해보겠습니다.

In [ ]:
submission = pd.read_csv('gender_submission.csv')

In [ ]:
submission.head()

In [ ]:
prediction = model.predict(X_test)
submission['Survived'] = prediction

In [ ]:
submission.to_csv('BCT_1st_submission.csv', index=False)